In [1]:
from typing import List
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import syllables
from collections import defaultdict
from pqdm.processes import pqdm
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from nltk.corpus import stopwords
import re
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
import pandas as pd
import os
import spacy
from numpy.linalg import norm
from tqdm import tqdm

import os
from sklearn.metrics.pairwise import cosine_distances


nlp = spacy.load("en_core_web_sm")
ESSAY_SET = 4

# Define QWK Metric functions

In [2]:
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the quadratic weighted kappa
    quadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.

    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.

    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.

    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator


def linear_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the linear weighted kappa
    linear_weighted_kappa calculates the linear weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.

    linear_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.

    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.

    linear_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = abs(i - j) / float(num_ratings - 1)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator


def kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the kappa
    kappa calculates the kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.

    kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.

    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.

    kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            if i == j:
                d = 0.0
            else:
                d = 1.0
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator


def mean_quadratic_weighted_kappa(kappas, weights=None):
    """
    Calculates the mean of the quadratic
    weighted kappas after applying Fisher's r-to-z transform, which is
    approximately a variance-stabilizing transformation.  This
    transformation is undefined if one of the kappas is 1.0, so all kappa
    values are capped in the range (-0.999, 0.999).  The reverse
    transformation is then applied before returning the result.

    mean_quadratic_weighted_kappa(kappas), where kappas is a vector of
    kappa values

    mean_quadratic_weighted_kappa(kappas, weights), where weights is a vector
    of weights that is the same size as kappas.  Weights are applied in the
    z-space
    """
    kappas = np.array(kappas, dtype=float)
    if weights is None:
        weights = np.ones(np.shape(kappas))
    else:
        weights = weights / np.mean(weights)

    # ensure that kappas are in the range [-.999, .999]
    kappas = np.array([min(x, .999) for x in kappas])
    kappas = np.array([max(x, -.999) for x in kappas])

    z = 0.5 * np.log((1 + kappas) / (1 - kappas)) * weights
    z = np.mean(z)
    return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)


def weighted_mean_quadratic_weighted_kappa(solution, submission):
    predicted_score = submission[submission.columns[-1]].copy()
    predicted_score.name = "predicted_score"
    if predicted_score.index[0] == 0:
        predicted_score = predicted_score[:len(solution)]
        predicted_score.index = solution.index
    combined = solution.join(predicted_score, how="left")
    groups = combined.groupby(by="essay_set")
    kappas = [quadratic_weighted_kappa(group[1]["essay_score"], group[1]["predicted_score"]) for group in groups]
    weights = [group[1]["essay_weight"].irow(0) for group in groups]
    return mean_quadratic_weighted_kappa(kappas, weights=weights)

In [3]:
df = pd.read_csv(
    f"./essay_set_{ESSAY_SET}_and_gpt_sent_embs.tsv",
    sep="\t",
    encoding="ISO-8859-1",
    usecols=["essay_id", "essay_set", "essay", "domain1_score", "domain2_score", "embs"],
)

In [4]:
from collections import Counter

Counter(df.domain1_score)

Counter({0: 311, 3: 253, 2: 570, 1: 636})

In [5]:
min_rating, max_rating = int(df.domain1_score.min()), int(df.domain1_score.max())

In [6]:
df.head()

,essay_id,essay_set,essay,domain1_score,domain2_score,embs
0,8863,4,The author concludes the story with this becau...,0,NaN,"[[0.06260137259960175, 0.0265642236918211, 0.0..."
1,8864,4,The narrater has that in with Paragraph becuse...,0,NaN,"[[0.012648290954530239, 0.04837891086935997, 0..."
2,8865,4,The author concludes the story with that passa...,3,NaN,"[[0.08485699445009232, -0.004821700509637594, ..."
3,8866,4,The author ended the story with this paragraph...,2,NaN,"[[0.06390519440174103, 0.017933126538991928, -..."
4,8867,4,The author concludes the story with this parag...,2,NaN,"[[0.04947877302765846, 0.014020605012774467, 0..."


In [7]:
sentence_embs = [eval(embs) for embs in df.embs.tolist()]

In [8]:
texts, scores = df.essay.tolist(), df.domain1_score.tolist()

In [9]:
text = texts[1]


def get_features(text: str, score: int, embeddings):
    
    
    # These are experimental, based on spatial measurements
    dist = cosine_distances(embeddings)
    nn_distances = np.min(dist + np.diag(np.diag(dist) + 10), axis=1)
    avg_nn_distance = np.mean(nn_distances)
    max_nn_distance = np.max(nn_distances)
    min_nn_distance = np.min(nn_distances)
    r_distance = 2*np.sqrt(dist.shape[0])*avg_nn_distance
    cum_freq_dist_nn_dist = np.mean(nn_distances <= avg_nn_distance)
    givenness = []
    for i in range(2, len(embeddings)):
        x = embeddings[0:i]
        u, s, vh = np.linalg.svd(x)
        orthonormal_vector = vh[-1]
        givenness.append(np.dot(embeddings[i], orthonormal_vector))

    avg_givenness = np.mean(givenness)
    max_givenness = np.max(givenness)
    min_givenness = np.min(givenness)
    givenness_proj = []
    for i in range(2, len(embeddings)):
        x = np.array(embeddings[0:i])
        A = x.T
        b = embeddings[i]
        c = np.linalg.lstsq(A, b, rcond=None)[0]
        bw = A.dot(c)
        bwo = b - bw
        N = np.dot(b, bwo)
        G = np.dot(b, bw)
        givenness_proj.append(N / (G + N))

    avg_givenness_proj = np.mean(givenness_proj)
    min_givenness_proj = np.min(givenness_proj)
    max_givenness_proj = np.max(givenness_proj)
    max_min_giv_ratio = max_givenness / min_givenness
    centroid = np.sum(embeddings, axis=0) / len(embeddings)
    dist_to_centroid = []
    for embedding in embeddings:
        dist_to_centroid.append(np.dot(centroid, embedding))

    avg_dist_to_centroid = np.mean(dist_to_centroid)
    max_dist_to_centroid = np.max(dist_to_centroid)
    min_dist_to_centroid = np.min(dist_to_centroid)
    std_distance = np.sqrt(np.sum(np.sum(
        (embeddings - centroid)**2, axis=1))/len(embeddings))
    relative_distance = std_distance / max_dist_to_centroid
    det_dist = np.linalg.det(dist)
    features = {}
    features.update({
        "avg_nn_distance": avg_nn_distance,
        "max_nn_distance": max_nn_distance,
        "min_nn_distance": min_nn_distance,
        "r_distance": r_distance,
        "cum_freq_dist_nn_dist": cum_freq_dist_nn_dist,
        "avg_givenness": avg_givenness,
        "max_givenness": max_givenness,
        "min_givenness": min_givenness,
        "max_min_giv_ratio": max_min_giv_ratio,
        "avg_givenness_proj": avg_givenness_proj,
        "min_givenness_proj": min_givenness_proj,
        "max_givenness_proj": max_givenness_proj,
        "max_min_giv_ratio": max_min_giv_ratio,
        "avg_dist_to_centroid": avg_dist_to_centroid,
        "max_dist_to_centroid": max_dist_to_centroid,
        "min_dist_to_centroid": min_dist_to_centroid,
        "std_distance": std_distance,
        "relative_distance": relative_distance,
        "det_dist": det_dist,
    })
    features["score"] = score

    return features

# print(get_features(texts[0], scores[0], sentence_embs[0]))
args = list(zip(texts, scores, sentence_embs))
train_result = pqdm(args, get_features, n_jobs=8, argument_type="args")

QUEUEING TASKS | :   0%|          | 0/1770 [00:00<?, ?it/s]

/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


PROCESSING TASKS | :   0%|          | 0/1770 [00:00<?, ?it/s]

/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/

/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/

...

/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/

/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/

/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/

/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dpalmasan/.pyenv/versions/3.8.10/lib/python3.8/

COLLECTING RESULTS | :   0%|          | 0/1770 [00:00<?, ?it/s]

In [10]:
train_results = []
for idx, res in enumerate(train_result):
    if type(res) != dict:
        continue
    train_results.append(res)
        
essay_set_features = pd.DataFrame(train_results).fillna(0)

In [11]:
X, y = essay_set_features.drop(["score"], axis=1), essay_set_features.score

In [12]:
clf = RandomForestRegressor(
    n_estimators=200, max_features=1/3, max_depth=12)

In [13]:
def essay_metrics(clf, X, y_true):
    y_pred = np.clip(np.round(clf.predict(X)), min_rating, max_rating)
    qwk = quadratic_weighted_kappa(y_true, y_pred, min_rating=min_rating, max_rating=max_rating)
    ea = np.sum(y_true == y_pred) / y_true.shape[0]
    aa = np.sum((y_true - y_pred) <= 1) / y_true.shape[0]
    return {
        "qwk": qwk,
        "ea": ea,
        "aa": aa
    }

In [14]:
assert min_rating == y.min()
assert max_rating == y.max()

# SC Model

In [15]:
from sklearn.model_selection import cross_validate

results = cross_validate(clf, X, y, scoring=essay_metrics, cv=10)

In [16]:
np.max(results["test_qwk"]), np.max(results["test_ea"]), np.max(results["test_aa"])

(0.5228648285137862, 0.5538461538461539, 1.0)